<a href="https://colab.research.google.com/github/OlanreleOladeji/Demand-forecast--XGBoost/blob/main/Products_quantity_forecasting_model_(XGB).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data upload and preprocessing

In [888]:
import pandas as pd

In [889]:
from google.colab import files

In [887]:
uploaded = files.upload()

Saving 1pharma-data.csv to 1pharma-data (4).csv


In [890]:
df = pd.read_csv('1pharma-data.csv')

In [891]:
df.head()

,Distributor,Customer Name,City,Country,Latitude,Longitude,Channel,Sub-channel,Product_ID,Product Class,Sales,Price,Revenue,Month,Year,Name of Sales Rep,Manager,Sales Team,Unnamed: 18
0,Gottlieb-Cruickshank,"Zieme, Doyle and Kunze",Lublin,Poland,51.2333,22.5667,Hospital,Private,Topipizole,Mood Stabilizers,4.0,368,1472.0,January,2018,Mary Gerrard,Britanny Bold,Delta,NaN
1,Gottlieb-Cruickshank,Feest PLC,Świecie,Poland,53.4167,18.4333,Pharmacy,Retail,Choriotrisin,Antibiotics,7.0,591,4137.0,January,2018,Jessica Smith,Britanny Bold,Delta,NaN
2,Gottlieb-Cruickshank,Medhurst-Beer Pharmaceutical Limited,Rybnik,Poland,50.0833,18.5000,Pharmacy,Institution,Acantaine,Antibiotics,30.0,66,1980.0,January,2018,Steve Pepple,Tracy Banks,Bravo,NaN
3,Gottlieb-Cruickshank,Barton Ltd Pharma Plc,Czeladź,Poland,50.3333,19.0833,Hospital,Private,Lioletine Refliruvax,Analgesics,6.0,435,2610.0,January,2018,Mary Gerrard,Britanny Bold,Delta,NaN
4,Gottlieb-Cruickshank,Keeling LLC Pharmacy,Olsztyn,Poland,53.7800,20.4942,Pharmacy,Retail,Oxymotroban Fexoformin,Analgesics,20.0,458,9160.0,January,2018,Anne Wu,Britanny Bold,Delta,NaN


Check dataset structure

In [892]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254082 entries, 0 to 254081
Data columns (total 19 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Distributor        254082 non-null  object 
 1   Customer Name      254082 non-null  object 
 2   City               254082 non-null  object 
 3   Country            254082 non-null  object 
 4   Latitude           254082 non-null  float64
 5   Longitude          254082 non-null  float64
 6   Channel            254082 non-null  object 
 7   Sub-channel        254082 non-null  object 
 8   Product_ID         254082 non-null  object 
 9   Product Class      254082 non-null  object 
 10  Sales              254082 non-null  float64
 11  Price              254082 non-null  int64  
 12  Revenue            254082 non-null  float64
 13  Month              254082 non-null  object 
 14  Year               254082 non-null  int64  
 15  Name of Sales Rep  254082 non-null  object 
 16  Ma

Data Exploration and Cleaning

Print the column headings

In [893]:
print(df.columns.tolist())


['Distributor', 'Customer Name', 'City', 'Country', 'Latitude', 'Longitude', 'Channel', 'Sub-channel', 'Product_ID', 'Product Class', 'Sales', 'Price', 'Revenue', 'Month', 'Year', 'Name of Sales Rep', 'Manager', 'Sales Team', 'Unnamed: 18']


Dropping columns

In [894]:
df= df.drop(['Channel', 'Sub-channel', 'Product Class', 'Name of Sales Rep', 'Price','Manager', 'Sales Team', 'Customer Name', 'City', 'Country', 'Distributor', 'Latitude', 'Longitude', 'Channel', 'Sub-channel', 'Product Class', 'Unnamed: 18'], axis=1)

In [895]:
df.head()

,Product_ID,Sales,Revenue,Month,Year
0,Topipizole,4.0,1472.0,January,2018
1,Choriotrisin,7.0,4137.0,January,2018
2,Acantaine,30.0,1980.0,January,2018
3,Lioletine Refliruvax,6.0,2610.0,January,2018
4,Oxymotroban Fexoformin,20.0,9160.0,January,2018


Aggregate by month and year Sum sales and revenue for each product at each level.

In [896]:
aggregated_monthly_revenue= df.groupby(['Year', 'Month', 'Product_ID'])['Sales'].sum().reset_index()


In [897]:
aggregated_monthly_sales = df.groupby(['Year', 'Month', 'Product_ID'])['Revenue'].sum().reset_index()

In [898]:
merged_df = pd.merge(aggregated_monthly_sales, aggregated_monthly_revenue, on=['Year', 'Month', 'Product_ID'])

In [899]:
merged_df.head()

,Year,Month,Product_ID,Revenue,Sales
0,2017,April,Abatatriptan,1046220.0,1410.0
1,2017,April,Abilovir Aprotasol,531006.0,1974.0
2,2017,April,Abobozolid,101250.0,1350.0
3,2017,April,Abranatal Lysoprosate,2565327.0,3767.0
4,2017,April,Abtasol,447876.0,594.0


In [900]:
merged_df.shape

(11520, 5)

In [901]:
df= merged_df

In [902]:
#print(df['Product_ID'].unique())

Feature Engineering

Split Dataset

In [903]:
from sklearn.model_selection import train_test_split

In [904]:
# Feature Column
X = ['Revenue']

# Target Column
y = ['Sales']

In [905]:
indices = df.index

In [906]:
train_df, test_df = train_test_split(indices, test_size=0.2, random_state=42)


In [907]:
train_df = df.loc[train_df].copy()
test_df = df.loc[test_df].copy()

In [908]:
train_X = train_df[X]
train_y = train_df[y]
test_X = test_df[X]
test_y = test_df[y]

In [909]:
display(train_df.head(10))

,Year,Month,Product_ID,Revenue,Sales
11473,2020,September,Sanctumine,187591.0,1299.0
782,2017,February,Bexridol,117680.0,1471.0
4080,2018,July,Abatatriptan,1835708.0,2474.0
1156,2017,January,Sevoderm,876486.0,1322.0
1536,2017,June,Dorzovorin Pentadarone,174988.0,1067.0
11049,2020,October,Acubulin,348390.0,711.0
5186,2018,November,Lovapur,1775322.0,3222.0
2534,2017,October,Ketamara Evogel,443520.0,730.0
10208,2020,June,Interfedox Pilobamol,14719.0,359.0
4609,2018,March,Apromin,496500.0,1324.0


In [910]:
display(test_df.head(10))

,Year,Month,Product_ID,Revenue,Sales
1078,2017,January,Halciramine,155829.0,381.0
9315,2020,December,Secrelazine Insonamic,646808.0,932.0
3299,2018,August,Pulmodiol Adalaxime,422290.0,698.0
7894,2019,May,Tetratanyl,2730784.0,5344.0
11037,2020,November,Zonixamine Immulinum,704770.0,2365.0
1142,2017,January,Quiniprosyn,360895.0,811.0
3833,2018,February,Xenaminphen,127920.0,656.0
4205,2018,July,Imiformin Transmunex,2035600.0,5089.0
9120,2020,December,Abatatriptan,402906.0,2743.0
4978,2018,May,Propratecan,1659988.0,2482.0


In [911]:
print('Training set: %s\nTest set: %s' % (train_df.shape, test_df.shape))

Training set: (9216, 5)
Test set: (2304, 5)


In [912]:
train_X

,Revenue
11473,187591.0
782,117680.0
4080,1835708.0
1156,876486.0
1536,174988.0
...,...
11284,1305174.0
5191,404855.0
5390,3921854.0
860,350366.0


In [913]:
train_y

,Sales
11473,1299.0
782,1471.0
4080,2474.0
1156,1322.0
1536,1067.0
...,...
11284,1731.0
5191,2165.0
5390,8563.0
860,1049.0


Dataset Scaling using MinMaxScaler

In [914]:
test_X

,Revenue
1078,155829.0
9315,646808.0
3299,422290.0
7894,2730784.0
11037,704770.0
...,...
7033,494592.0
1833,2884492.0
5882,276422.0
8332,315360.0


In [915]:
test_y

,Sales
1078,381.0
9315,932.0
3299,698.0
7894,5344.0
11037,2365.0
...,...
7033,1104.0
1833,5746.0
5882,3371.0
8332,1460.0


In [916]:
from sklearn.preprocessing import MinMaxScaler

Scaling the train_X and test_X :

In [917]:
scaler_X = MinMaxScaler()

In [918]:
scaler_X.fit(train_X)

MinMaxScaler()

In [919]:
train_X_Scaled = scaler_X.transform(train_X)

In [920]:
train_X_Scaled = pd.DataFrame(train_X_Scaled, columns=X, index=train_X.index)

In [921]:
#train_X_Scaled.head()

In [922]:
test_X_Scaled = scaler_X.transform(test_X)

In [923]:
test_X_Scaled = pd.DataFrame(test_X_Scaled, columns=X, index=test_X.index)

In [924]:
#test_X_Scaled.head()

Scaling the train_y and test_y :

In [925]:
scaler_y = MinMaxScaler()

In [926]:
scaler_y.fit(train_y)

MinMaxScaler()

In [927]:
train_y_Scaled = scaler_y.transform(train_y)

In [928]:
train_y_Scaled = pd.DataFrame(train_y_Scaled, columns=y, index=train_y.index)

In [929]:
#train_y_Scaled.head()

In [930]:
test_y_Scaled = scaler_y.transform(test_y)

In [931]:
test_y_Scaled = pd.DataFrame(test_y_Scaled, columns=y, index=test_y.index)

In [932]:
#test_y_Scaled.head(10)

XGBoost Algorithm

In [933]:
from xgboost import XGBRegressor, DMatrix

In [934]:
model = XGBRegressor()

In [935]:
model.fit(train_X_Scaled, train_y_Scaled)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

Make predictions on the test dataset (test_X_Scaled):

In [936]:
forecast = model.predict(test_X_Scaled)

In [937]:
print(forecast)

[0.01352304 0.01482592 0.01230563 ... 0.01165497 0.0124406  0.01148146]


Summarize progress

In [938]:
Progress_summary_df = pd.DataFrame({
        'Revenue': test_X_Scaled['Revenue'],
        'Actual': test_y_Scaled['Sales'],
        'forecast': forecast[:test_y_Scaled.shape[0]].flatten()
})

print("Progress summary:")

display(Progress_summary_df.head(5))

Progress summary:


,Revenue,Actual,forecast
1078,0.034981,0.002758,0.013523
9315,0.041353,0.007418,0.014826
3299,0.038439,0.005439,0.012306
7894,0.068402,0.044737,0.039625
11037,0.042106,0.019539,0.015432


Evaluate model with performance

In [939]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error

In [940]:
mean_squared_error(test_y_Scaled, forecast)

0.0004245069285389036

In [941]:
mean_absolute_error(test_y_Scaled, forecast)

0.008594419807195663

In [942]:
root_mean_squared_error(test_y_Scaled, forecast)

0.020603565499186516

Inverse transformation

Transform prediction

In [943]:
forecast_value = scaler_y.inverse_transform(forecast.reshape(-1, 1))

Transform revenue (test_X) to original values

In [944]:
test_X = scaler_X.inverse_transform(test_X_Scaled)

In [945]:
Revenue_value = pd.DataFrame(test_X, columns=['Revenue'])

Transform the test_y_Scaled back to the original scale (test_y)

In [946]:
test_y = scaler_y.inverse_transform(test_y_Scaled)

In [947]:
Actual_value = pd.DataFrame(test_y, columns=['Sales'])

Summary of the actual test_y and forecast value transformation

In [948]:
Inverse_Summary_df = pd.DataFrame({
        'Actual_value': Actual_value['Sales'],
        'forecast_value': forecast_value[:Actual_value.shape[0]].flatten(),
        'Revenue_value': Revenue_value['Revenue']
})

print("Summary:")

display(Inverse_Summary_df.head())

Summary:


,Actual_value,forecast_value,Revenue_value
0,381.0,1653.734009,155829.0
1,932.0,1807.764526,646808.0
2,698.0,1509.808594,422290.0
3,5344.0,4739.577148,2730784.0
4,2365.0,1879.421021,704770.0


Weighted Root Mean Squared Scaled Error

Forecast summary with product identifiers

In [949]:
test_df_subset = test_df[['Year', 'Month', 'Product_ID']].reset_index(drop=True)

summary = Inverse_Summary_df.merge(
    test_df_subset,
    left_index=True,
    right_index=True,
    how='left'
)

display(summary.head())

,Actual_value,forecast_value,Revenue_value,Year,Month,Product_ID
0,381.0,1653.734009,155829.0,2017,January,Halciramine
1,932.0,1807.764526,646808.0,2020,December,Secrelazine Insonamic
2,698.0,1509.808594,422290.0,2018,August,Pulmodiol Adalaxime
3,5344.0,4739.577148,2730784.0,2019,May,Tetratanyl
4,2365.0,1879.421021,704770.0,2020,November,Zonixamine Immulinum


Calculating Root Mean Squared Scaled Error (RMSSE)

In [950]:
def rmsse(train_y, test_y, forecast):
    forecast_mse = np.mean((Actual_value - forecast_value)**2, axis=1)
    train_mse = np.mean(np.diff(train_y)**2)

    return np.sqrt(forecast_mse / train_mse)

rmsse_vals = rmsse(train_y['Sales'].values, Actual_value, forecast_value)
print(rmsse_vals)


0       0.259067
1       0.178263
2       0.165245
3       0.123031
4       0.098840
          ...   
2299    0.094633
2300    0.060636
2301    0.394506
2302    0.013387
2303    0.052389
Length: 2304, dtype: float64


In [951]:
print('Number of values: %d' % len(rmsse_vals))
print('RMSSE mean: %.3f' % np.mean(rmsse_vals))

Number of values: 2304
RMSSE mean: 0.207


In [952]:
revenue_total = revenue_lm = test_X[:, 0].sum()

In [953]:
Revenue_weight = revenue_lm = test_X[:, 0] / revenue_total.sum()

In [954]:
print(Revenue_weight)

[6.62498255e-05 2.74986794e-04 1.79534225e-04 ... 1.17519263e-04
 1.34073536e-04 1.05573981e-04]


In [955]:
print('Number of weights: %s' % len(Revenue_weight))

Number of weights: 2304


In [956]:
sum_of_weights = Revenue_weight.sum()
print('sum_of_weights:', Revenue_weight.sum())

sum_of_weights: 1.0


In [957]:
wrmsse = (Revenue_weight * rmsse_vals).sum()
print('WRMSSE: %.3f' % wrmsse)

WRMSSE: 0.546


Hyperparemeter tuning

In [958]:
from sklearn.model_selection import GridSearchCV

In [959]:
param_grid = {
    "n_estimators": [200, 400, 600, 800, 1000],
    "learning_rate": [0.01, 0.1],
    "max_depth": [3, 7],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0],
    "gamma": [0, 0.1],
   }

In [960]:
grid_search = GridSearchCV(estimator= XGBRegressor(), param_grid= param_grid, cv=5, verbose=1)

In [961]:
grid_search.fit(train_X_Scaled, train_y_Scaled)

Fitting 5 folds for each of 160 candidates, totalling 800 fits


GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             param_grid={'colsample_bytree': [0.8, 1.0], 'gamma': [0, 0.1],
                         'learning_rate': [0.01, 0.1], 'max_depth': [3, 7],
                         'n_estimators': [200, 400, 600, 800, 1000],
                         'subsample': [0.8, 1.0]},
             verbose=1)

Best parameters

In [962]:
grid_search.best_params_

{'colsample_bytree': 0.8,
 'gamma': 0,
 'learning_rate': 0.01,
 'max_depth': 3,
 'n_estimators': 800,
 'subsample': 0.8}

Make predictions on the test dataset (test_X_Scaled) using grid search best estimator

In [978]:
grid_search.best_params_
{'colsample_bytree': 0.8,
 'gamma': 0,
 'learning_rate': 0.01,
 'max_depth': 3,
 'n_estimators': 800,
 'subsample': 0.8}

{'colsample_bytree': 0.8,
 'gamma': 0,
 'learning_rate': 0.01,
 'max_depth': 3,
 'n_estimators': 800,
 'subsample': 0.8}

In [979]:
forecast_tuned = grid_search.best_estimator_.predict(test_X_Scaled)

In [980]:
print(forecast_tuned)

[0.01120172 0.01478454 0.0129205  ... 0.01153177 0.01238332 0.01150333]


In [981]:
tuned_progress_summary_df = pd.DataFrame({
        'Revenue': test_X_Scaled['Revenue'],
        'Actual': test_y_Scaled['Sales'],
        'forecast_tuned': forecast_tuned[:test_y_Scaled.shape[0]].flatten()
})

print("tuned_progress summary:")

display(tuned_progress_summary_df.head(5))

tuned_progress summary:


,Revenue,Actual,forecast_tuned
1078,0.034981,0.002758,0.011202
9315,0.041353,0.007418,0.014785
3299,0.038439,0.005439,0.012921
7894,0.068402,0.044737,0.040470
11037,0.042106,0.019539,0.015504


Summarize progress after tunning

Evaluate model with performance after tunning

In [982]:
mean_squared_error(test_y_Scaled, forecast_tuned)

0.00042260016198270023

In [983]:
mean_absolute_error(test_y_Scaled, forecast_tuned)

0.008524362929165363

In [984]:
root_mean_squared_error(test_y_Scaled, forecast_tuned)

0.020557241514325142

In [985]:
forecast_tuned = scaler_y.inverse_transform(forecast_tuned.reshape(-1, 1))


In [986]:
tuned_inverse_summary_df = pd.DataFrame({
        'Actual_value': Actual_value['Sales'],
        'forecast_tuned': forecast_tuned[:Actual_value.shape[0]].flatten(),
        'Revenue_value': Revenue_value['Revenue']
})

print("Summary:")

display(tuned_inverse_summary_df.head())

Summary:


,Actual_value,forecast_tuned,Revenue_value
0,381.0,1379.301270,155829.0
1,932.0,1802.872559,646808.0
2,698.0,1582.500488,422290.0
3,5344.0,4839.471191,2730784.0
4,2365.0,1887.928223,704770.0


In [990]:
def rmsse_tuned(train_y, test_y, forecast_tuned):
    forecast_tuned_mse = np.mean((Actual_value - forecast_tuned)**2, axis=1)
    train_mse = np.mean(np.diff(train_y)**2)

    return np.sqrt(forecast_tuned_mse / train_mse)

rmsse_tuned = rmsse_tuned(train_y['Sales'].values, Actual_value, forecast_tuned)
print(rmsse_tuned)



0       0.203206
1       0.177267
2       0.180041
3       0.102698
4       0.097109
          ...   
2299    0.112627
2300    0.090890
2301    0.397471
2302    0.012008
2303    0.052915
Length: 2304, dtype: float64


In [988]:
wrmsse_tuned = (Revenue_weight * rmsse2_vals).sum()
print('WRMSSE_tuned: %.3f' % wrmsse_tuned)

WRMSSE_tuned: 0.544
